# Sample / Load SD data from PAR model for evaluation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from IPython.utils import io

In [ ]:
from sdv.sequential import PARSynthesizer

## Load SD generator & sample or load SD

In [ ]:
# False to load SD, True to load model and generate
generate = False
# it should be 4000 but it takes 35 minutes on a GPU
num_seq = 4000
days = 1
# real data dir
data_dir = "../"
# generate_tsne
generate_tsne = False

In [ ]:
%%time
if generate:
    synthesizer = PARSynthesizer.load(f"quick_test_PAR_full_cols_{days}_days.pkl")
    #synthesizer.verbose = False
    with io.capture_output() as captured:
        synthetic_data = synthesizer.sample(num_sequences=num_seq);
    synthetic_data.to_csv(f"synthetic_data_sdv_{days}_days.csv")
else:
    synthetic_data = pd.read_csv(f"synthetic_data_sdv_{days}_days.csv", index_col=0)

## Load RD

In [ ]:
real_data = pd.read_csv(f"{data_dir}real_data_sdv_{days}_days.csv", index_col=0)

In [ ]:
real_data

In [ ]:
len(real_data.columns)

### Revert data types for SD

In [ ]:
#sd_df = synthetic_data

In [ ]:
# check types are equal
synthetic_data.dtypes.equals(real_data.dtypes)

In [ ]:
# check columns are equal
synthetic_data.columns.equals(real_data.columns)

In [ ]:
synthetic_data = synthetic_data.convert_dtypes()
real_data = real_data.convert_dtypes()

In [ ]:
# check types are equal
synthetic_data.dtypes.equals(real_data.dtypes)

In [ ]:
# check columns are equal
synthetic_data.columns.equals(real_data.columns)

In [ ]:
real_data.describe()

In [ ]:
synthetic_data.describe()

In [ ]:
# get number of unique ids
udids = len(synthetic_data.datapoint_id.unique())

In [ ]:
print(f"Unique datapoint_ids = {udids}")

In [ ]:
# let's throw and error if real and SD data do not have same number of buildings
if udids != len(real_data.datapoint_id.unique()):
    raise ValueError("Real and SD have different number of buildings")

In [ ]:
from synthcity import metrics
from synthcity.benchmark import Benchmarks
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader

### Convert to TimeSeriesDataLoaders
- Convert dataframes to synthcity data loaders for plotting

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [ ]:
def convert_to_tsdloaders(df, ct=None):
    group_timeseries = df[["datapoint_id", "energy_elec", "energy_gas"]].groupby('datapoint_id', sort=False)
    timeseries_dfs = [group_timeseries.get_group(t)[["energy_elec", "energy_gas"]].reset_index(drop=True) for t in group_timeseries.groups]
    #timeseries_dfs = [df[["energy_elec", "energy_gas"]][i:i+days*24].reset_index(drop=True).copy() for i in range(0, df.shape[0], days*24)]
   
    static_df = df.drop(columns=["Timestamp", "energy_elec", "energy_gas"]).drop_duplicates().reset_index(drop=True)
    #static_df["datapoint_id"]=static_df.index
    #static_df = df.drop(columns=["datapoint_id"])
    if not ct:
        ct = make_column_transformer((OrdinalEncoder(), make_column_selector(dtype_include="string")),
                                     ("passthrough",make_column_selector(dtype_exclude=["string"])))
                                                                                                    #                               , dtype_include=["Float64", "Int64"])))
    column_order = list(static_df.select_dtypes(include=["string"]).columns) + list(static_df.select_dtypes(include=["Float64", "Int64"]).columns)
    tr_df = pd.DataFrame(ct.fit_transform(static_df), index=static_df.index, columns=column_order)[static_df.columns]
    
    
    observation_data = []
    outcome = []
    for tdf in timeseries_dfs:
        observations = list(tdf.index)
        observation_data.append(observations)
        outcome.append(1)    
    loader = []
    loader = TimeSeriesDataLoader(
         temporal_data=timeseries_dfs,
         observation_times=observation_data,
         static_data=tr_df,#static_df,
         outcome=pd.DataFrame(outcome),#pd.DataFrame(btap_in_df[["Total Energy"]])#moutcome),
    )
    return loader, tr_df, timeseries_dfs, observation_data, ct

In [ ]:
%%time
rd_loader, static_df, timeseries_dfs, observation_data, ct = convert_to_tsdloaders(real_data)#[["datapoint_id", ":dcv_type", "Timestamp", "energy_elec", "energy_gas"]])

In [ ]:
len(rd_loader.static_features)

In [ ]:
%%time
sd_loader, sd_static_df, sd_timeseries_dfs, sd_observation_data, sd_ct = convert_to_tsdloaders(synthetic_data, ct)#[["datapoint_id", ":dcv_type", "Timestamp", "energy_elec", "energy_gas"]], ct)

## Evaluation
- It takes 2 1/2 hours to run metrics on cpu (3 minutes on GPU): sanity, stats, and privacy
- We need to check if the metrics are being computed accordingly. The plots are right because we are using the SD dataloaders.

In [ ]:
metrics_dict = {
    'sanity': ['data_mismatch', 'common_rows_proportion', 'nearest_syn_neighbor_distance', 'close_values_probability', 'distant_values_probability'],
    'stats': ['jensenshannon_dist', 'chi_squared_test', 'feature_corr', 'inv_kl_divergence', 'ks_test', 'max_mean_discrepancy', 'wasserstein_dist', 'prdc', 'alpha_precision', 'survival_km_distance'],
    # these do not make sense
    #'performance': ['linear_model', 'mlp', 'xgb', 'feat_rank_distance'],
    #'detection': ['detection_xgb', 'detection_mlp', 'detection_gmm', 'detection_linear'],
    # this takes a long time to run
    'privacy': ['delta-presence', 'k-anonymization', 'k-map', 'distinct l-diversity', 'identifiability_score']
}

In [ ]:
%%time
scores = []
for k, v in metrics_dict.items():
    print(f"{k} metrics")
    score = metrics.Metrics.evaluate(real_data, synthetic_data, metrics={k: v})
    scores.append(score)
    #print(score)

### Synthcity can compare several batches of generated SD and output statistics from the metrics for each evaluation. Here we only care about generating metrics for one batch.

Thus in the `scores` dictionary the key `mean` has a different meaning, it reflects the value of the metric. All other keys can be discarded.

In [ ]:
scores[0][['mean']]

In [ ]:
scores[1][['mean']]

In [ ]:
scores[2][['mean']]

## Plots

- Distributions
- tSNE
- Time Series comparison???

In [ ]:
from synthcity.metrics.plots import plot_marginal_comparison, plot_tsne

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
plot_marginal_comparison(plt, rd_loader, sd_loader)

In [ ]:
%%time
if generate_tsne:
    plot_tsne(plt, rd_loader, sd_loader)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from synthcity.metrics.plots import plot_marginal_comparison, plot_tsne

In [ ]:
import random

#### Time Series plots

In [ ]:
# Set the style for the plots
sns.set_style("whitegrid")
sns.set_context("paper")  # Sets the scaling of elements such as the font size

# Formatter for the y-axis
def thousands(x, pos):
    'The two args are the value and tick position'
    return f'{x * 1e-3:,.0f}k'.replace(',', ' ')

formatter = FuncFormatter(thousands)

# Specify the datapoint_ids you want to plot
real_specific_id = None
synthetic_specific_id = None


# If specific IDs are not provided, choose a random one from the unique values
if not real_specific_id:
    real_specific_id = random.randint(0, num_seq-1)
if not synthetic_specific_id:
    synthetic_specific_id = random.randint(0, num_seq-1)

# Electric Energy Comparison
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
time = [str(i+1) for i in range(24)]
ax.plot(time, timeseries_dfs[real_specific_id]['energy_elec'], label='Real Data - Electric Energy', color='#0072B2', linewidth=2.5)
ax.plot(time, sd_timeseries_dfs[synthetic_specific_id]['energy_elec'], label='Synthetic Data - Electric Energy', color='#D55E00', linestyle='--', linewidth=2.5)
#ax.yaxis.set_major_formatter(formatter)
plt.title('Comparison of Electric Energy Usage Over Time', fontsize=16)
plt.xlabel('Timestamp', fontsize=14)
plt.ylabel('Electric Energy Usage (kWh)', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
sns.despine(left=True)
plt.tight_layout()

# Save the plot with high resolution
#plt.savefig('electric_energy_comparison.png', dpi=300)  # Replace with your path

plt.show()

In [ ]:
# Gas Energy Comparison
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
ax.plot(time, timeseries_dfs[real_specific_id]['energy_gas'], label='Real Data - Gas Energy', color='#0072B2', linewidth=2.5)
ax.plot(time, sd_timeseries_dfs[synthetic_specific_id]['energy_gas'], label='Synthetic Data - Gas Energy', color='#D55E00', linestyle='--', linewidth=2.5)
#ax.yaxis.set_major_formatter(formatter)
plt.title('Comparison of Gas Energy Usage Over Time', fontsize=16)
plt.xlabel('Timestamp', fontsize=14)
plt.ylabel('Gas Energy Usage (kWh)', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
sns.despine(left=True)
plt.tight_layout()

# Save the plot with high resolution
#plt.savefig('gas_energy_comparison.png', dpi=300)  # Replace with your path

plt.show()

In [ ]:
synthetic_data.columns

In [ ]:
#Mean Energy Usage 
real_data_avg = real_data.groupby('Timestamp').mean().reset_index()
synthetic_data_avg = synthetic_data.groupby('Timestamp').mean().reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
ax.plot(time, real_data_avg['energy_elec'], label='Real Data - Average Electric Energy', color='#0072B2', linewidth=2.5)
ax.plot(time, synthetic_data_avg['energy_elec'], label='Synthetic Data - Average Electric Energy', color='#D55E00', linestyle='--', linewidth=2.5)
ax.yaxis.set_major_formatter(formatter)
ax.set_title('Average Comparison of Electric Energy Usage Over Time', fontsize=16)
ax.set_xlabel('Hour of the day', fontsize=14)
ax.set_ylabel('Electric Energy Usage (Joules per hour)', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
sns.despine(left=True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
ax.plot(time, real_data_avg['energy_gas'], label='Real Data - Average Gas Energy', color='#0072B2', linewidth=2.5)
ax.plot(time, synthetic_data_avg['energy_gas'], label='Synthetic Data - Average Gas Energy', color='#D55E00', linestyle='--', linewidth=2.5)
ax.yaxis.set_major_formatter(formatter)
plt.title('Average Comparison of Gas Energy Usage Over Time', fontsize=16)
plt.xlabel('Hour of the day', fontsize=14)
plt.ylabel('Gas Energy Usage (Joules per hour)', fontsize=14)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
sns.despine(left=True)
plt.tight_layout()
plt.show()

In [ ]:
%%time
#Mean Energy Usage 
real_data_std = real_data.groupby('Timestamp').std().reset_index()
synthetic_data_std = synthetic_data.groupby('seq_time_id').std().reset_index()

### Add peaks and valleys

Find max and min on the time series and add it to the static columns, discarding time series data

In [ ]:
real_data

In [ ]:
def pick_peaks(df, count=1):
    """ Select max and min values for each time series and add it to a dataframe along with the timestamp

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with static and time series values.
    count : int
        The number of max and min values to grab.

    Returns
    -------
    pd.DataFrame
        DataFrame with static features and mins and maxes with timestamps
    """
    # get static features
    static_df = df.drop(columns=["Timestamp", "energy_elec", "energy_gas"]).drop_duplicates().reset_index(drop=True)

    # get timeseries for each utility
    group_timeseries_elec = df[["datapoint_id", "Timestamp", "energy_elec"]].groupby('datapoint_id', sort=False)
    group_timeseries_gas = df[["datapoint_id", "Timestamp", "energy_gas"]].groupby('datapoint_id', sort=False)

    # emaxmins ={f"temax{i}": [] for i in range(count)} | {f"emax{i}": [] for i in range(count)} |\
    #           {f"temin{i}": [] for i in range(count)} | {f"emin{i}": [] for i in range(count)}
    
    # create a dict to store maxes and mins, change this to adjust for a particular order
    emaxmins = {}
    for i in range(count):
            emaxmins[f"temax_{i}"] = []
            emaxmins[f"emax_{i}"] = []
            emaxmins[f"temin_{i}"] = []
            emaxmins[f"emin_{i}"] = []
    
    for t in group_timeseries_elec.groups:
        df_ts = group_timeseries_elec.get_group(t)[["Timestamp", "energy_elec"]]
        ts = df_ts.energy_elec
        # locate mins and maxes and append it to emaxmins dict
        for i in range(count):
            imaxv, maxv = ts.idxmax(), ts.max()
            iminv, minv = ts.idxmin(), ts.min()
            max_ts = df_ts.loc[imaxv, "Timestamp"]
            min_ts = df_ts.loc[iminv, "Timestamp"]
            emaxmins[f"temax_{i}"].append(max_ts)
            emaxmins[f"emax_{i}"].append(maxv)
            emaxmins[f"temin_{i}"].append(min_ts)
            emaxmins[f"emin_{i}"].append(minv)
            ts = ts.drop(imaxv)
            ts = ts.drop(iminv)

    # create a new dataframe with mins and maxes and their timestamp
    edfp = pd.DataFrame(emaxmins)

    # create a dict to store maxes and mins, change this to adjust for a particular order
    gmaxmins = {}
    for i in range(count):
            gmaxmins[f"tgmax_{i}"] = []
            gmaxmins[f"gmax_{i}"] = []
            gmaxmins[f"tgmin_{i}"] = []
            gmaxmins[f"gmin_{i}"] = []
    
    for t in group_timeseries_gas.groups:
        df_ts = group_timeseries_gas.get_group(t)[["Timestamp", "energy_gas"]]#.reset_index(drop=True)
        ts = df_ts.energy_gas

        # Note 1: dont look for minmax if timeseries is flat or zero
        # often there is no gas measurements
        # Note 2: this doesnt handle all cases, this assumes that if there is a gas measurement
        # then there are measurements for each hour.
        # Note 3: mins and maxes for gas contributes to sparsity given that a lot of 
        # buildings do not have gas consumption.
        if len(ts.unique()) > 2:
            for i in range(count):
                imaxv, maxv = ts.idxmax(), ts.max()
                iminv, minv = ts.idxmin(), ts.min()
                max_ts = df_ts.loc[imaxv, "Timestamp"]
                min_ts = df_ts.loc[iminv, "Timestamp"]
                gmaxmins[f"tgmax_{i}"].append(max_ts)
                gmaxmins[f"gmax_{i}"].append(maxv)
                gmaxmins[f"tgmin_{i}"].append(min_ts)
                gmaxmins[f"gmin_{i}"].append(minv)
                ts = ts.drop(imaxv)
                ts = ts.drop(iminv)
        else:
            for i in range(count):
                gmaxmins[f"tgmax_{i}"].append(str(datetime.strptime("2000-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')))
                gmaxmins[f"gmax_{i}"].append(0)
                gmaxmins[f"tgmin_{i}"].append(str(datetime.strptime("2000-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')))
                gmaxmins[f"gmin_{i}"].append(0)
            
    
    gdfp = pd.DataFrame(gmaxmins)

    # concatenate static_df with min and maxes dataframes
    df_ = pd.concat([static_df, edfp, gdfp], axis=1)

    # Quick validation, the static emax0 values should be the same as computing
    # the max of each time series.
    # TODO: Validation needs to be done for all cases.
    # if not np.allclose(df_.emax_0.unique(), df[["datapoint_id","energy_elec"]].groupby('datapoint_id', sort=False).max().values.flatten()):
    #    raise ValueError("Max of time series mismatch")
        
    # let's reorder the columns
    cols = list(df_.columns)
    lencs = len(cols)
    print(f"Columns in {len(df.columns)}, columns out {lencs}")
    #return df_, edfp
    if lencs != len(df.columns) + 8*count - 3:
        raise ValueError("Input / output columns mismatch")
    
    return df_[cols[0:1]+cols[-8*count:]+cols[1:lencs-8*count]]


In [ ]:
%%time
real_data_peaks = pick_peaks(real_data)

In [ ]:
real_data_peaks

## Done!